# Signal Portfolio - Data Analytics for Finance

**Signal Name (e.g., Book to Market):** Gross Profit scaled by Assets (gp_at)

In [ ]:
//Load Libraries

#r "nuget: FSharp.Data"
#r "nuget: FSharp.Stats"
#r "nuget: Plotly.NET,2.0.0-preview.17"
#r "nuget: Plotly.NET.Interactive,2.0.0-preview.17"
#r "nuget: DiffSharp-lite"
#r "nuget: Accord"
#r "nuget: Accord.Statistics"

#load "Portfolio.fsx"
#load "Common.fsx"
#load "YahooFinance.fsx"

open DiffSharp
open System
open FSharp.Data
open FSharp.Stats
open Plotly.NET
open Portfolio
open Common
open Accord
open Accord.Statistics.Models.Regression.Linear
open YahooFinance


Microsoft.DotNet.Interactive.InstallPackagesMessage


In [ ]:
// Set dotnet interactive formatter to plaintext
Formatter.Register(fun (x:obj) (writer: TextWriter) -> fprintfn writer "%120A" x )
Formatter.SetPreferredMimeTypesFor(typeof<obj>, "text/plain")
// Make plotly graphs work with interactive plaintext formatter
Formatter.SetPreferredMimeTypesFor(typeof<GenericChart.GenericChart>,"text/html")

In [ ]:
let [<Literal>] ResolutionFolder = __SOURCE_DIRECTORY__
Environment.CurrentDirectory <- ResolutionFolder

let [<Literal>] MySignalFilePath = "data/gp_at.csv"
//let [<Literal>] myExcessReturnPortfoliosPath = "data/myExcessReturnPortfolios.csv"
let [<Literal>] IdAndReturnsFilePath = "data/id_and_return_data.csv"

let strategyName = "gp_at"


In [ ]:
type IdAndReturnsType = 
    CsvProvider<Sample=IdAndReturnsFilePath,
                // The schema parameter is not required,
                // but I am using it to override some column types
                // to make filtering easier.
                // If I didn't do this these particular columns 
                // would have strings of "1" or "0", but explicit boolean is nicer.
                Schema="obsMain(string)->obsMain=bool,exchMain(string)->exchMain=bool",
                ResolutionFolder=ResolutionFolder>

type MySignalType = 
    CsvProvider<MySignalFilePath,
                ResolutionFolder=ResolutionFolder>

let idAndReturnsCsv = IdAndReturnsType.GetSample()
let idAndReturnsRows = idAndReturnsCsv.Rows |> Seq.toList

let mySignalCsv = MySignalType.GetSample()
let mySignalRows = mySignalCsv.Rows |> Seq.toList

## 3.1 Overview

In [ ]:
 mySignalRows
|> List.map (fun row -> row.Id)
|> List.distinct
|> List.length

In [ ]:
// Analysis of the signal

// Number of stocks per month
let countMySignalRows (rows: list<MySignalType.Row>) = 
    let monthly =
        mySignalRows
        |> List.groupBy (fun row -> row.Eom)
        |> List.sortBy (fun (month, rows) -> month)
    [ for (month, rows) in monthly do
        let nStocks = 
            rows
            |> List.map (fun row -> row.Id)
            |> List.distinct
            |> List.length
        month, nStocks ]

let stockPerMonthCounts =
    let ColumnChart = 
        mySignalRows
        |> countMySignalRows
    Chart.Column(ColumnChart, Name = "Stocks per month")
        |> Chart.withXAxisStyle (TitleText="Month")
        |> Chart.withYAxisStyle (TitleText="Number of Stocks")

stockPerMonthCounts

In [ ]:
type NonMissingSignal =
    {
        Id: string
        Eom: DateTime
        Signal: float
    }

let myNonMissingSignals =
    mySignalRows
    |> List.choose (fun row -> 
        match row.Signal with
        | None -> None
        | Some signal -> 
            Some { Id = row.Id; Eom = row.Eom; Signal = signal })

let countMyNonMissingSignalRows (rows: list<NonMissingSignal>) =
    let by_month =
        rows
        |> List.groupBy (fun row -> row.Eom)
        |> List.sortBy (fun (month, rows) -> month)
    [ for (month, rows) in by_month do
        let nr_of_stocks = 
            rows
            |> List.map (fun row -> row.Id)
            |> List.distinct
            |> List.length
        month, nr_of_stocks ]

let stock_counts_nonmissing =
    let to_plot = 
        myNonMissingSignals
        |> countMyNonMissingSignalRows
    Chart.Column(to_plot, Name = "Non Missing Stocks")
    |> Chart.withXAxisStyle (TitleText="Month")
    |> Chart.withYAxisStyle (TitleText="Number of Stocks")

stock_counts_nonmissing

In [ ]:
let NonMissingData =
    myNonMissingSignals 
    |> List.map(fun x -> x.Signal)
    |> List.toArray

//Minimum
let Minimum =
    myNonMissingSignals
    |> List.map(fun x -> x.Signal)
    |> List.min

// Maximum
let Maximum =
    myNonMissingSignals
    |> List.map(fun x -> x.Signal)
    |> List.max

// Median
let Median =
    myNonMissingSignals
    |> List.map(fun x -> x.Signal)
    |> List.median

// Standard Deviation
let StDev = 
    myNonMissingSignals
    |> List.map(fun x -> x.Signal)
    |> Seq.stDev

// Average
let Average =
    myNonMissingSignals
    |> List.map(fun x -> x.Signal)
    |> List.average


printfn "Minimum is equal to %A" Minimum
printfn "Maximum is equal to %A" Maximum
printfn "Median is equal to %A" Median
printfn "Standard Deviation is equal to %A" StDev
printfn "Average is equal to %A" Average

let signalP01: float = Quantile.compute 0.01 NonMissingData
let signalP10: float = Quantile.compute 0.1 NonMissingData
let signalP50: float = Quantile.compute 0.5 NonMissingData
let signalP90: float = Quantile.compute 0.9 NonMissingData
let signalP99: float = Quantile.compute 0.99 NonMissingData

printfn "1st percentile of the non-missing signal is equal to %A" signalP01
printfn "10th percentile of the non-missing signal is equal to %A" signalP10
printfn "50th percentile of the non-missing signal is equal to %A" signalP50
printfn "90th percentile of the non-missing signal is equal to %A" signalP90
printfn "99th percentile of the non-missing signal is equal to %A" signalP99

In [ ]:
let winsorizeSignals (signalOb: NonMissingSignal) =
    let newSignal =
        if signalOb.Signal < signalP01 then 
            signalP01
        elif signalOb.Signal > signalP99 then
            signalP99
        else
            signalOb.Signal
    { signalOb with Signal = newSignal }

let myWinsorizedSignals =
    myNonMissingSignals
    |> List.map winsorizeSignals

let byStockMonthSignals: list<DateTime * list<NonMissingSignal>> =
    myWinsorizedSignals
    |> List.groupBy(fun x -> DateTime(x.Eom.Year, x.Eom.Month, 1))

byStockMonthSignals

In [ ]:

let byStockMonthIdAndReturnMap: Map<string * DateTime, IdAndReturnsType.Row> =
    idAndReturnsRows
    |> List.map(fun x ->
        let ym = x.Eom
        let key = id x.Id, ym
        key, x)
    |> Map

byStockMonthIdAndReturnMap.Keys

let signals_winsorized_smallcap =
    [ for i in myWinsorizedSignals do 
        match byStockMonthIdAndReturnMap |> Map.find (i.Id,i.Eom) with
        | x when x.SizeGrp = "small" && x.Eom.Year = 2015 -> Some i.Signal
        | y -> None]
    |> List.choose id
signals_winsorized_smallcap
    |> Chart.Histogram

In [ ]:
let signals_winsorized_largecap =
    [ for i in myWinsorizedSignals do 
        match byStockMonthIdAndReturnMap |> Map.find (i.Id,i.Eom) with
        | x when x.SizeGrp = "large" && x.Eom.Year = 2015 -> Some i.Signal
        | y -> None]
    |> List.choose id
signals_winsorized_largecap
    |> Chart.Histogram

In [ ]:
type SortedPort =
    { Portfolio: int
      Eom: DateTime
      Stocks: list<NonMissingSignal> }

let terciles =
    byStockMonthSignals
    |> List.collect (fun (eom, signals) ->
        let sortedSignals =
            signals
            |> List.sortBy (fun signalOb -> signalOb.Signal)
            |> List.splitInto 3
        sortedSignals
        |> List.mapi (fun i p -> 
            { Portfolio = i + 1
              Eom = eom
              Stocks = p }))

let portfolio_terciles =
    terciles
    |> List.groupBy (fun row -> row.Portfolio)

//Portfolio 1
let portfolio_1 = 
    snd portfolio_terciles[0]
    |> List.map (fun x -> x.Stocks)

let avg_portfolio_1 = 
    [for x in portfolio_1 do
        let m =
            x
            |> List.map (fun row -> row.Eom)
            |> List.distinct
        let avg_signal =
            x
            |> List.map (fun row -> row.Signal)
            |> List.average
        m[0], avg_signal]
    |> List.sort

//Portfolio 2
let portfolio_2 = 
    snd portfolio_terciles[1]
    |> List.map (fun x -> x.Stocks)

let avg_portfolio_2= 
    [for x in portfolio_2 do
        let m =
            x
            |> List.map (fun row -> row.Eom)
            |> List.distinct
        let avg_signal =
            x
            |> List.map (fun row -> row.Signal)
            |> List.average
        m[0], avg_signal]
    |> List.sort

//Portfolio 3
let portfolio_3 = 
    snd portfolio_terciles[2]
    |> List.map (fun x -> x.Stocks)

let avg_portfolio_3 = 
    [for x in portfolio_3 do
        let m =
            x
            |> List.map (fun row -> row.Eom)
            |> List.distinct
        let avg_signal =
            x
            |> List.map (fun row -> row.Signal)
            |> List.average
        m[0], avg_signal]
    |> List.sort

// Combined Chart
Chart.combine (
    [ Chart.Line(avg_portfolio_1, Name="1st Portfolio")
      Chart.Line(avg_portfolio_2, Name="2nd Portfolio")
      Chart.Line(avg_portfolio_3, Name="3rd Portfolio")])

## 3.2 Strategy Analysis 

### Construct the Strategy

In [ ]:
let msfBySecurityIdAndMonth =
    idAndReturnsRows
    |> List.map(fun row -> 
        let id = Other row.Id
        let month = DateTime(row.Eom.Year,row.Eom.Month,1)
        let key = id, month
        key, row)
    |> Map    

let signalBySecurityIdAndMonth =
    mySignalRows
    |> List.choose(fun row -> 
        match row.Signal with
        | None -> None 
        | Some signal ->
            let id = Other row.Id
            let month = DateTime(row.Eom.Year,row.Eom.Month,1)
            let key = id, month
            Some (key, signal))
    |> Map

let securitiesByFormationMonth =
    idAndReturnsRows
    |> List.groupBy(fun x -> DateTime(x.Eom.Year, x.Eom.Month,1))
    |> List.map(fun (ym, obsThisMonth) -> 
        let idsThisMonth = [ for x in obsThisMonth do Other x.Id ]
        ym, idsThisMonth)
    |> Map

let getInvestmentUniverse formationMonth =
    match Map.tryFind formationMonth securitiesByFormationMonth with
    | Some securities -> 
        { FormationMonth = formationMonth 
          Securities = securities }
    | None -> failwith $"{formationMonth} is not in the date range"


// High signal of gp_at predicts high returns, no need to multiply for -1
let getMySignal (securityId, formationMonth) =
    match Map.tryFind (securityId, formationMonth) signalBySecurityIdAndMonth with
    | None -> None
    | Some signal ->
        Some { SecurityId = securityId
               Signal = signal }

// For the whole investment universe
let getMySignals (investmentUniverse: InvestmentUniverse) =
    let listOfSecuritySignals =
        investmentUniverse.Securities
        |> List.choose(fun security -> 
            getMySignal (security, investmentUniverse.FormationMonth))    
    
    { FormationMonth = investmentUniverse.FormationMonth 
      Signals = listOfSecuritySignals }

// My market capitalization
let getMarketCap (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> None
    | Some row -> 
        match row.MarketEquity with
        | None -> None
        | Some me -> Some (security, me)

let getSecurityReturn (security, formationMonth) =
    // If the security has a missing return, assume that we got 0.0.
    // Note: If we were doing excess returns, we would need 0.0 - rf.
    let missingReturn = 0.0
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> security, missingReturn
    | Some x ->  
        match x.Ret with 
        | None -> security, missingReturn
        | Some r -> security, r

// Restrictions
let isObsMain (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.ObsMain

let isPrimarySecurity (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.PrimarySec

let isCommonStock (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.Common

let isExchMain (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.ExchMain

let hasMarketEquity (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.MarketEquity.IsSome

let myFilters securityAndFormationMonth =
    isObsMain securityAndFormationMonth &&
    isPrimarySecurity securityAndFormationMonth &&
    isCommonStock securityAndFormationMonth &&
    isExchMain securityAndFormationMonth &&
    isExchMain securityAndFormationMonth &&
    hasMarketEquity securityAndFormationMonth

let doMyFilters (universe:InvestmentUniverse) =
    let filtered = 
        universe.Securities
        // my filters expect security, formationMonth
        |> List.map(fun security -> security, universe.FormationMonth)
        // do the filters
        |> List.filter myFilters
        // now convert back from security, formationMonth -> security
        |> List.map fst
    { universe with Securities = filtered }

let formStrategy ym =
    ym
    |> getInvestmentUniverse
    |> doMyFilters
    |> getMySignals
    |> assignSignalSort strategyName 3
    |> List.map (giveValueWeights getMarketCap)
    |> List.map (getPortfolioReturn getSecurityReturn)  

// Define Sample months

let startSample = 
    idAndReturnsRows
    |> List.map(fun row -> DateTime(row.Eom.Year,row.Eom.Month,1))
    |> List.min

let endSample = 
    let lastMonthWithData = 
        idAndReturnsRows
        |> Seq.map(fun row -> DateTime(row.Eom.Year,row.Eom.Month,1))
        |> Seq.max
    // The end of sample is the last month when we have returns.
    // So the last month when we can form portfolios is one month
    // before that.
    lastMonthWithData.AddMonths(-1) 

let sampleMonths = getSampleMonths (startSample, endSample)

// My strategy Portfolio 

let doParallel = true
let portfolios =
    if doParallel then
        sampleMonths
        |> List.toArray
        |> Array.Parallel.map formStrategy
        |> Array.toList
        |> List.collect id
    else
        sampleMonths
        |> List.collect formStrategy

Form a long-short strategy portfolio that is long your top portfolio and short your bottom portfolio.

In [ ]:
let ff3 = French.getFF3 Frequency.Monthly
let monthlyRiskFreeRate =
    [ for obs in ff3 do 
        let key = DateTime(obs.Date.Year,obs.Date.Month,1)
        key, obs.Rf ]
    |> Map

let portfolioExcessReturns =
    portfolios
    |> List.map(fun x -> 
        match Map.tryFind x.YearMonth monthlyRiskFreeRate with 
        | None -> failwith $"Can't find risk-free rate for {x.YearMonth}"
        | Some rf -> { x with Return = x.Return - rf })

let long = 
    portfolioExcessReturns 
    |> List.filter(fun x -> 
        x.PortfolioId = Indexed {| Name = strategyName ; Index = 3 |})

let short = 
    portfolioExcessReturns 
    |> List.filter(fun x -> 
        x.PortfolioId = Indexed {| Name = strategyName; Index = 1 |})

let longShort = 
    let shortByYearMonthMap = 
        short 
        |> List.map(fun row -> row.YearMonth, row) 
        |> Map
    
    [ for longObs in long do
        match Map.tryFind longObs.YearMonth shortByYearMonthMap with
        | None -> failwith "probably your date variables are not aligned for a weird reason"
        | Some shortObs ->
            { PortfolioId = Named "Long-Short gp_at"
              YearMonth = longObs.YearMonth
              Return = longObs.Return - shortObs.Return } ] 

// Plot
let cumulateSimpleReturn (xs: PortfolioReturn list) =
    let accumulator (priorObs:PortfolioReturn) (thisObs:PortfolioReturn) =
        let asOfNow = (1.0 + priorObs.Return)*(1.0 + thisObs.Return) - 1.0
        { thisObs with Return = asOfNow}
    match xs |> List.sortBy(fun x -> x.YearMonth) with
    | [] -> []     
    | head::tail -> 
        (head, tail) 
        ||> List.scan accumulator

let longshortCumulative = longShort |> cumulateSimpleReturn

let longshortCumulativeChart =
    longshortCumulative
    |> List.map(fun x -> x.YearMonth, x.Return)
    |> Chart.Line 
    |> Chart.withTitle "Growth of 1 Euro"

In [ ]:
longshortCumulativeChart

### Analyze the performance of your long-only and long-short portfolios.
Plot cumulative returns for your two portfolios and for the excess returns of the value-weighted stock market portfolio (from the Ken
French data, MktRf) in one chart. You should make two graphs:
- one graph showing cumulative returns for all the portfolios.
- one graph showing cumulative returns with a constant leverage
applied to each portfolio so that they all have an annualized
volatility of 10% over the full sample.

In [ ]:
// Add value weighted portfolio with same time range

let portfolioReturnPlot (xs:PortfolioReturn list) =
    xs
    |> List.map(fun x -> x.YearMonth, x.Return)
    |> Chart.Line 
    |> Chart.withTitle "Cumulative Returns"

let vwMktRf =
    let portfolioMonths = 
        portfolioExcessReturns 
        |> List.map(fun x -> x.YearMonth)
    let minYm = portfolioMonths |> List.min
    let maxYm = portfolioMonths |> List.max
    
    [ for x in ff3 do
        if x.Date >= minYm && x.Date <= maxYm then
            { PortfolioId = Named("Mkt-Rf")
              YearMonth = x.Date
              Return = x.MktRf } ]


// Plot cumulative returns for your two portfolios 
// and for the excess returns of the value-weighted stock market portfolio

// One graph with all the portfolio
let combinedChart =
    List.concat [long; longShort; vwMktRf]
    |> List.groupBy(fun x -> x.PortfolioId)
    |> List.map(fun (portId, xs) ->
        xs
        |> cumulateSimpleReturn
        |> portfolioReturnPlot
        |> Chart.withTraceInfo (Name=portId.ToString())
        |> Chart.withTitle "Cumulative Returns"
        |> Chart.withSize(900,600))
    |> Chart.combine

combinedChart

In [ ]:
// One graph showing cumulative returns with a constant leverage applied to each 
// portfolio so that they all have an annualized volatility of 10% over the full sample.

let annualizeMonthlyStdDev monthlyStdDev: float = sqrt(12.0) * monthlyStdDev
let get_stdevAnnualized (input: PortfolioReturn list) = 
    input
    |> Seq.stDevBy (fun x -> x.Return)
    |> annualizeMonthlyStdDev

let stdevLongAnnualized = get_stdevAnnualized long
let stdevLongShortAnnualized = get_stdevAnnualized longShort
let stdevVwMktRfAnnualized = get_stdevAnnualized vwMktRf


let leverage_fun input: float =
    0.1 / input
let leverageLong = leverage_fun stdevLongAnnualized
let leverageLongShort = leverage_fun stdevLongShortAnnualized
let leverageVwMktRf = leverage_fun stdevVwMktRfAnnualized

let long10_initial = 
    long
    |> List.map(fun (x) ->
        { PortfolioId = x.PortfolioId;
          YearMonth = x.YearMonth;
          Return = leverageLong * x.Return })

let long10 = 
    long10_initial
    |> List.map (fun x -> 
        { PortfolioId = Named "Gp_at Long 10% Stdev"
          YearMonth = x.YearMonth
          Return = x.Return })

let longShort10_initial = 
    longShort
    |> List.map(fun (x) ->
        { PortfolioId = Named "Gp_at Short 10% Stdev";
          YearMonth = x.YearMonth;
          Return = leverageLongShort * x.Return })

let longShort10 = 
    longShort10_initial
    |> List.map (fun x -> 
        { PortfolioId = Named "Gp_at Long-Short 10% StdDev"
          YearMonth = x.YearMonth
          Return = x.Return })

let vwMktRf10_initial = 
    vwMktRf
    |> List.map(fun (x) ->
        { PortfolioId = x.PortfolioId;
          YearMonth = x.YearMonth;
          Return = leverageVwMktRf * x.Return })

let vwMktRf10 = 
    vwMktRf10_initial
    |> List.map (fun x -> 
        { PortfolioId = Named "Mkt-Rf 10% StdDev"
          YearMonth = x.YearMonth
          Return = x.Return })

let combinedChart_lev10 =
    List.concat [long10; longShort10; vwMktRf10]
    |> List.groupBy(fun x -> x.PortfolioId)
    |> List.map(fun (portId, xs) ->
        xs
        |> cumulateSimpleReturn
        |> portfolioReturnPlot
        |> Chart.withTraceInfo (Name=portId.ToString())
        |> Chart.withTitle "Cumulative Returns levered"
        |> Chart.withSize(900,600))
    |> Chart.combine

combinedChart_lev10

Create a table to report performance measures for your long-only,
long-short, and value-weighted market portfolios for the first half of the sample, 
the second half, and the full period.
It is not necessary to report results for the 10% volatility versions in this table. 

For each period report:
For the long-only and long-short portfolios:
- What is their average annualized excess return?
- What are their annualized Sharpe ratios?
- What are their CAPM and Fama-French 3-factor alphas and t-statistics for these alphas?
- What are their information ratios?

For the value-weighted market portfolio:
- What is the average annualized excess return? 
- What are the annualized Sharpe ratio?

In [ ]:
let first_half (lst:list<'a>) = 
    let len = List.length lst
    let mid = int(len / 2)
    let first_list = lst |> Seq.take mid |> Seq.toList
    first_list

let second_half (lst:list<'a>) = 
    let len = List.length lst
    let mid = int(len / 2)
    let second_list = lst |> Seq.skip mid |> Seq.toList
    second_list

In [ ]:
// Split the sample
let longFirstHalf = (first_half long)
let longSecondHalf = (second_half long)

let longshortFirstHalf = (first_half longShort)
let longshortSecondHalf = (second_half longShort)

let vwMktFirstHalf = (first_half vwMktRf)
let vwMkktSecondHalf = (second_half vwMktRf)

// Average Annualized Excess Return 
let annualizeMonthlyReturns monthlyReturn = 12.0 * monthlyReturn

let ann_avg (input:PortfolioReturn List) = 
    let value = 
        input
        |> List.map (fun value -> value.Return)
        |> List.map ( fun value -> annualizeMonthlyReturns value)
        |> List.average
    let y = value * 100.0
    y

let long_avg = ann_avg long
let long_FirstHalf_avg =  ann_avg longFirstHalf
let long_SecondHalf_avg =  ann_avg longSecondHalf

let longShort_avg = ann_avg longShort
let longShort_FirstHalf_avg = ann_avg longshortFirstHalf
let longShort_SecondHalf_avg = ann_avg longshortSecondHalf

let vwMktRf_avg = ann_avg  vwMktRf
let vwMktRf_FirstHalf_avg = ann_avg  vwMktFirstHalf 
let vwMktRf_SecondHalf_avg = ann_avg  vwMkktSecondHalf


// Annualized Sharpe Ratio

let Sharpe (xs: float seq) =
    (Seq.mean xs) / (Seq.stDev xs)

let annualizeMonthlySharpe monthlySharpe = sqrt(12.0) * monthlySharpe

let ann_sharpe (input: PortfolioReturn List) = 
    input
    |> List.map (fun x -> x.Return)
    |> Sharpe
    |> annualizeMonthlySharpe

let long_sharpe = ann_sharpe long
let long_FirstHalf_sharpe =  ann_sharpe longFirstHalf
let long_SecondHalf_sharpe =  ann_sharpe longSecondHalf

let longShort_sharpe = ann_sharpe longShort
let longShort_FirstHalf_sharpe = ann_sharpe longshortFirstHalf
let longShort_SecondHalf_sharpe = ann_sharpe longshortSecondHalf

let vwMktRf_sharpe = ann_sharpe  vwMktRf
let vwMktRf_FirstHalf_sharpe = ann_sharpe vwMktFirstHalf 
let vwMktRf_SecondHalf_sharpe = ann_sharpe vwMkktSecondHalf



In [ ]:
let header = ["";"Avg Annualized Excess Return"]

let rows = [
                ["Long"; string long_avg];
                ["Long 1st half"; string long_FirstHalf_avg];
                ["Long 2nd half"; string long_SecondHalf_avg];
                ["Long-Short"; string longShort_avg];
                ["Long-Short 1st half"; string longShort_FirstHalf_avg];
                ["Long-Short 2nd half"; string longShort_SecondHalf_avg];
                ["MktRf"; string vwMktRf_avg];
                ["MktRf 1st half"; string vwMktRf_FirstHalf_avg];
                ["MktRf 2nd half"; string vwMktRf_SecondHalf_avg]
]

Chart.Table(header, 
            rows) 
    |> Chart.withSize (800, 400)

In [ ]:
let header = ["";"Sharpe Ratio"]

let rows = [
                ["Long"; string long_sharpe ];
                ["Long 1st half"; string long_FirstHalf_sharpe ];
                ["Long 2nd half"; string long_SecondHalf_sharpe];
                ["Long-Short"; string longShort_sharpe];
                ["Long-Short 1st half"; string longShort_FirstHalf_sharpe];
                ["Long-Short 2nd half";string longShort_SecondHalf_sharpe];
                ["MktRf"; string vwMktRf_sharpe;  ];
                ["MktRf 1st half"; string vwMktRf_FirstHalf_sharpe];
                ["MktRf 2nd half"; string vwMktRf_SecondHalf_sharpe]
]

Chart.Table(header, 
            rows) 
    |> Chart.withSize (800, 400)

In [ ]:
// Alpha, tstat and Information Ratio
type RegData =
    { Date : DateTime
      Portfolio : float
      MktRf : float 
      Hml : float 
      Smb : float }

let ff3ByMonth = 
    ff3
    |> Array.map(fun x -> DateTime(x.Date.Year, x.Date.Month,1), x)
    |> Map

let reg_data (input: PortfolioReturn list) = 
    input 
    |> List.map(fun port ->
        let month_list = DateTime(port.YearMonth.Year,port.YearMonth.Month,1)
        match Map.tryFind month_list ff3ByMonth with
        | None -> failwith "probably you messed up your days of months"
        | Some ff3 -> 
            { Date = month_list
              Portfolio = port.Return
              MktRf = ff3.MktRf 
              Hml = ff3.Hml 
              Smb = ff3.Smb })
    |> List.toArray

let capmModelData (input: RegData array) = 
    input
    |> Array.map(fun obs -> [|obs.MktRf|], obs.Portfolio)
    |> Array.unzip 

let ff3ModelData (input: RegData array) =
    input
    |> Array.map(fun obs -> [|obs.MktRf; obs.Hml; obs.Smb |], obs.Portfolio)
    |> Array.unzip

type RegressionOutput =
    { Model : MultipleLinearRegression 
      TValuesWeights : float array
      TValuesIntercept : float 
      R2: float }

type XY = (float array) array * float array

let fitModel (x: (float array) array, y: float array) =
    let ols = new OrdinaryLeastSquares(UseIntercept=true)
    let estimate = ols.Learn(x,y)
    let mse = estimate.GetStandardError(x,y)
    let se = estimate.GetStandardErrors(mse, ols.GetInformationMatrix())
    let tvaluesWeights = 
        estimate.Weights
        |> Array.mapi(fun i w -> w / se.[i])
    let tvalueIntercept = estimate.Intercept / (se |> Array.last)
    let r2 = estimate.CoefficientOfDetermination(x,y)
    { Model = estimate
      TValuesWeights = tvaluesWeights
      TValuesIntercept = tvalueIntercept  
      R2 = r2 }

let capmEstimate (input: PortfolioReturn list) =
    reg_data input
    |> capmModelData
    |> fitModel

let ff3Estimate (input: PortfolioReturn list) = 
    reg_data input
    |> ff3ModelData
    |> fitModel

// Information Ratios

type Prediction = { Label : float; Score : float}

let makePredictions 
    (estimate:MultipleLinearRegression) 
    (x: (float array) array, y: float array) =
    (estimate.Transform(x), y)
    ||> Array.zip
    |> Array.map(fun (score, label) -> { Score = score; Label = label })

let residuals (xs: Prediction array) = xs |> Array.map(fun x -> x.Label - x.Score)

let informationRatio monthlyAlpha (monthlyResiduals: float array) =
    let annualAlpha = 12.0 * monthlyAlpha
    let annualStDev = sqrt(12.0) * (Seq.stDev monthlyResiduals)
    annualAlpha / annualStDev

let capmModelData_reg (input: PortfolioReturn list) =
    reg_data input
    |> capmModelData

let ff3ModelData_reg (input: PortfolioReturn list) =
    reg_data input
    |> ff3ModelData

let capmInformationRatio (input: PortfolioReturn list)  =
    let x =
        makePredictions (capmEstimate input).Model (capmModelData_reg input)
        |> residuals
    informationRatio (capmEstimate input).Model.Intercept x

let ff3InformationRatio (input: PortfolioReturn list)  =
    let x =
        makePredictions (ff3Estimate input).Model (ff3ModelData_reg input)
        |> residuals
    informationRatio (ff3Estimate input).Model.Intercept x

In [ ]:
// For the long portfolio

// CAPM
let capmEstimate_long1 = capmEstimate longFirstHalf
let capmEstimate_long2 = capmEstimate longSecondHalf
let capmEstimate_long = capmEstimate long

let capmAlpha_long1 = 12.0 * capmEstimate_long1.Model.Intercept
let capmAlpha_long2 = 12.0 * capmEstimate_long2.Model.Intercept 
let capmAlpha_long = 12.0 * capmEstimate_long.Model.Intercept 

let capmTstat_long1 = 12.0 * capmEstimate_long1.TValuesIntercept
let capmTstat_long2 = 12.0 * capmEstimate_long2.TValuesIntercept
let capmTstat_long = 12.0 * capmEstimate_long.TValuesIntercept

let capmInformationratio_long1 = capmInformationRatio longFirstHalf
let capmInformationratio_long2 = capmInformationRatio longSecondHalf
let capmInformationratio_long = capmInformationRatio long

// FF3
let ff3Estimate_long1 = ff3Estimate longFirstHalf
let ff3Estimate_long2 = ff3Estimate longSecondHalf
let ff3Estimate_long = ff3Estimate long

let ff3Alpha_long1 = 12.0 * ff3Estimate_long1.Model.Intercept 
let ff3Alpha_long2 = 12.0 * ff3Estimate_long2.Model.Intercept 
let ff3Alpha_long = 12.0 * ff3Estimate_long.Model.Intercept

let ff3Tstat_long1 = 12.0 * ff3Estimate_long1.TValuesIntercept
let ff3Tstat_long2 = 12.0 * ff3Estimate_long2.TValuesIntercept 
let ff3Tstat_long = 12.0 * ff3Estimate_long.TValuesIntercept

let ff3Informationratio_long1 = ff3InformationRatio longFirstHalf
let ff3Informationratio_long2 = ff3InformationRatio longSecondHalf
let ff3Informationratio_long = ff3InformationRatio long


In [ ]:
// Alphas and tstat for long-short portfolio

// CAPM
let capmEstimate_longshort1 = capmEstimate longshortFirstHalf
let capmEstimate_longshort2 = capmEstimate longshortSecondHalf
let capmEstimate_longshort = capmEstimate longShort

let capmAlpha_longshort1 = 12.0 * capmEstimate_longshort1.Model.Intercept
let capmAlpha_longshort2 = 12.0 * capmEstimate_longshort2.Model.Intercept 
let capmAlpha_longshort = 12.0 * capmEstimate_longshort.Model.Intercept 

let capmTstat_longshort1 = 12.0 * capmEstimate_longshort1.TValuesIntercept
let capmTstat_longshort2 = 12.0 * capmEstimate_longshort2.TValuesIntercept
let capmTstat_longshort = 12.0 * capmEstimate_longshort.TValuesIntercept

let capmInformationratio_longshort1 = capmInformationRatio longshortFirstHalf
let capmInformationratio_longshort2 = capmInformationRatio longshortSecondHalf
let capmInformationratio_longshort = capmInformationRatio longShort

// FF3
let ff3Estimate_longshort1 = ff3Estimate longshortFirstHalf
let ff3Estimate_longshort2 = ff3Estimate longshortSecondHalf
let ff3Estimate_longshort = ff3Estimate longShort

let ff3Alpha_longshort1 = 12.0 * ff3Estimate_longshort1.Model.Intercept 
let ff3Alpha_longshort2 = 12.0 * ff3Estimate_longshort2.Model.Intercept 
let ff3Alpha_longshort = 12.0 * ff3Estimate_longshort.Model.Intercept

let ff3Tstat_longshort1 = 12.0 * ff3Estimate_longshort1.TValuesIntercept
let ff3Tstat_longshort2 = 12.0 * ff3Estimate_longshort2.TValuesIntercept 
let ff3Tstat_longshort = 12.0 * ff3Estimate_longshort.TValuesIntercept

let ff3Informationratio_longshort1 = ff3InformationRatio longshortFirstHalf
let ff3Informationratio_longshort2 = ff3InformationRatio longshortSecondHalf
let ff3Informationratio_longshort = ff3InformationRatio longShort

In [ ]:
let header = [""; "CAPM Alpha"; "CAPM t-stat"; "CAPM IR"]

let rows = [
                ["Long"; string capmAlpha_long; string capmTstat_long; string capmInformationratio_long];
                ["Long 1st half"; string capmAlpha_long1; string capmTstat_long1; string capmInformationratio_long1 ];
                ["Long 2nd half"; string capmAlpha_long2; string capmTstat_long2; string capmInformationratio_long2 ];

                ["Long-Short"; string capmAlpha_longshort; string capmTstat_longshort; string capmInformationratio_longshort ];
                ["Long-Short 1st half"; string capmAlpha_longshort1; string capmTstat_longshort1; string capmInformationratio_longshort1];
                ["Long-Short 2nd half"; string capmAlpha_longshort2; string capmTstat_longshort2; string capmInformationratio_longshort2]
]

Chart.Table(header, 
            rows) 
    |> Chart.withSize (1000, 400)

In [ ]:
let header = [""; "FF3 Alpha"; "FF3 t-stat"; "FF3 IR"]

let rows = [
                ["Long"; string ff3Alpha_long; string ff3Tstat_long; string ff3Informationratio_long ];
                ["Long 1st half"; string ff3Alpha_long1; string ff3Tstat_long1; string ff3Informationratio_long1];
                ["Long 2nd half"; string ff3Alpha_long2; string ff3Tstat_long2; string ff3Informationratio_long2];

                ["Long-Short"; string ff3Alpha_longshort; string ff3Tstat_longshort; string ff3Informationratio_longshort ];
                ["Long-Short 1st half"; string ff3Alpha_longshort1; string ff3Tstat_longshort1; string ff3Informationratio_longshort1];
                ["Long-Short 2nd half"; string ff3Alpha_longshort2; string ff3Tstat_longshort2; string ff3Informationratio_longshort2]
                ]

Chart.Table(header,
            rows) 
    |> Chart.withSize (1000, 400)


## 3.3 Strategy as part of a diversified portfolio

### Identify the tangency portfolio

In [ ]:
type StockData =
    { Symbol : string 
      Date : DateTime
      Return : float }

// Gett ff3 asset pricing model data and transform to a StockData record type
let ff3new = French.getFF3 Frequency.Monthly |> Array.toList

let ff3StockData =
    [ 
       ff3new |> List.map(fun x -> {Symbol="HML";Date=x.Date;Return=x.Hml})
       ff3new |> List.map(fun x -> {Symbol="MktRf";Date=x.Date;Return=x.MktRf})
       ff3new |> List.map(fun x -> {Symbol="Smb";Date=x.Date;Return=x.Smb})
    ] |> List.concat

// Factor data
let tickers = 
    [ 
        "VTI" // Vanguard Total Stock Market ETF
        "BND" // Vanguard Total Bond Market ETF
    ]

let tickPrices = 
    YahooFinance.PriceHistory(
        tickers,
        startDate = DateTime(2000,1,1),
        interval = Monthly)

tickPrices

In [ ]:
// Returns from price Observations
let pricesToReturns (symbol, adjPrices: list<PriceObs>) =
    adjPrices
    |> List.sortBy (fun x -> x.Date)
    |> List.pairwise
    |> List.map (fun (day0, day1) ->
        let r = day1.AdjustedClose / day0.AdjustedClose - 1.0 
        { Symbol = symbol 
          Date = day1.Date 
          Return = r })

let tickReturns =
    tickPrices
    |> List.groupBy (fun x -> x.Symbol)
    |> List.collect pricesToReturns

Convert to excess returns

In [ ]:
let rf = Map [ for x in ff3 do x.Date, x.Rf ]
let initial_date = tickReturns[0].Date

let standardInvestmentsExcess_fun (input: StockData list) =
    let fin_value = 
        let append_1 =
            let maxff3Date = ff3new |> List.map(fun x -> x.Date) |> List.max
            tickReturns
            |> List.filter(fun x -> x.Date <= maxff3Date)
            |> List.map(fun x -> 
                match Map.tryFind x.Date rf with 
                | None -> failwith $"why isn't there a rf for {x.Date}"
                | Some rf -> { x with Return = x.Return - rf })
        let append_2 =
            let maxff3Date = ff3new |> List.map(fun x -> x.Date) |> List.max
            input 
            |> List.filter(fun x -> x.Date >= initial_date)
            |> List.filter(fun x -> x.Date <= maxff3Date)
            |> List.map(fun x -> 
                match Map.tryFind x.Date rf with 
                | None -> failwith $"why isn't there a rf for {x.Date}"
                | Some rf -> { x with Return = x.Return })
        append_1 @ append_2
        |> Seq.distinct
        |> List.ofSeq
    fin_value

let long_new = 
    long
    |> List.map(fun i ->
        { Symbol = "Long"
          Date = i.YearMonth
          Return = i.Return })  

let longshort_new = 
    longShort
    |> List.map(fun i ->
        { Symbol = "Long-Short"
          Date = i.YearMonth
          Return = i.Return })  

In [ ]:
let standardInvestmentsExcessLong = standardInvestmentsExcess_fun long_new
let standardInvestmentsExcessLongShort = standardInvestmentsExcess_fun longshort_new

In [ ]:
// Put stocks in a map keyed by symbol
let stockData_fun (input: StockData list) = 
    input
    |> List.groupBy(fun x -> x.Symbol)
    |> Map

In [ ]:
let stockDataLong = stockData_fun standardInvestmentsExcessLong
let stockDataLongShort = stockData_fun standardInvestmentsExcessLongShort

- One version using your long-only portfolio + other asset
- Another version using your long-short portfolio + other assets.

In [ ]:
let tickers_long =
    [ 
        "VTI" // Vanguard Total Stock Market ETF
        "BND" // Vanguard Total Bond Market ETF
        "Long"
    ]

let tickers_LongShort =
    [ 
        "VTI" // Vanguard Total Stock Market ETF
        "BND" // Vanguard Total Bond Market ETF
        "Long-Short"
    ]

let tickers_6040 =
    [ 
        "VTI" // Vanguard Total Stock Market ETF
        "BND" // Vanguard Total Bond Market ETF
    ]

 When estimating covariances/correlations, use the mutually overlapping time period

In [ ]:
let getCov xId yId (stockData: Map<string,StockData list>) =
    let xRet = 
        stockData[xId] 
        |> List.map (fun x -> x.Date,x.Return) 
        |> Map
    let yRet = 
        stockData[yId]
        |> List.map (fun y -> y.Date, y.Return)
        |> Map
    let overlappingDates =
        [ xRet.Keys |> set
          yRet.Keys |> set]
        |> Set.intersectMany
    [ for date in overlappingDates do xRet[date], yRet[date]]
    |> Seq.covOfPairs


In [ ]:
let covariances_long =
    [ for rowTick in tickers_long do 
        [ for colTick in tickers_long do
            getCov rowTick colTick stockDataLong ]]
    |> dsharp.tensor

covariances_long

In [ ]:
let covariances_longShort =
    [ for rowTick in tickers_LongShort do 
        [ for colTick in tickers_LongShort do
            getCov rowTick colTick stockDataLongShort ]]
    |> dsharp.tensor

covariances_longShort

In [ ]:
let means_long =
    [ for ticker in tickers_long do 
        stockDataLong[ticker]
        |> List.averageBy (fun x -> x.Return)]
    |> dsharp.tensor

let l_w' = dsharp.solve(covariances_long,means_long)
let wLong = l_w' / l_w'.sum()

let weights_long =
    Seq.zip tickers_long (wLong.toArray1D<float>())
    |> Map.ofSeq


In [ ]:
let means_longShort =
    [ for ticker in tickers_LongShort do 
        stockDataLongShort[ticker]
        |> List.averageBy (fun x -> x.Return)]
    |> dsharp.tensor

let ls_w' = dsharp.solve(covariances_longShort,means_longShort)
let wLongShort = ls_w' / ls_w'.sum()

let weights_longShort =
    Seq.zip tickers_LongShort (wLongShort.toArray1D<float>())
    |> Map.ofSeq

### Form comparison diversified portfolios. 
You must form a 60/40 portfolio that is invested 60% in the Vanguard Total Stock Market ETF (VTI) and 40% in the Vanguard Total Bond Market ETF (BND). Make sure you use excess returns. You may add other comparison portfolios if you wish, but this is not required.

In [ ]:
let stockDataByDate_fun (input: seq<list<StockData>>) = 
    input
    |> Seq.toList
    |> List.collect id 
    |> List.groupBy(fun x -> x.Date) 
    |> List.sortBy fst 

In [ ]:
let stockDataByDateLong = stockDataByDate_fun stockDataLong.Values
let stockDataByDateLongShort = stockDataByDate_fun stockDataLongShort.Values

In [ ]:
let allAssetsStart_long =
    stockDataByDateLong
    |> List.find(fun (month, stocks) -> stocks.Length = tickers_long.Length)
    |> fst 

let allAssetsEnd_long =
    stockDataByDateLong
    |> List.findBack(fun (month, stocks) -> stocks.Length = tickers_long.Length)
    |> fst

let allAssetsStart_longShort =
    stockDataByDateLongShort
    |> List.find(fun (month, stocks) -> stocks.Length = tickers_LongShort.Length)
    |> fst 

let allAssetsEnd_longShort =
    stockDataByDateLongShort
    |> List.findBack(fun (month, stocks) -> stocks.Length = tickers_LongShort.Length)
    |> fst

let stockDataByDateComplete_long =
    stockDataByDateLong
    |> List.filter(fun (date, stocks) -> 
        date >= allAssetsStart_long &&
        date <= allAssetsEnd_long)

let stockDataByDateComplete_longShort =
    stockDataByDateLongShort
    |> List.filter(fun (date, stocks) -> 
        date >= allAssetsStart_longShort &&
        date <= allAssetsEnd_longShort)

In [ ]:
let checkOfCompleteDataLong =
    stockDataByDateComplete_long
    |> List.map snd
    |> List.filter(fun x -> x.Length <> tickers_long.Length) // discard rows where we have all symbols.

if not (List.isEmpty checkOfCompleteDataLong) then 
        failwith "stockDataByDateComplete has months with missing stocks"


let checkOfCompleteData_longShort =
    stockDataByDateComplete_longShort
    |> List.map snd
    |> List.filter(fun x -> x.Length <> tickers_LongShort.Length) // discard rows where we have all symbols.

if not (List.isEmpty checkOfCompleteData_longShort) then 
        failwith "stockDataByDateComplete has months with missing stocks"

In [ ]:
checkOfCompleteDataLong

In [ ]:
checkOfCompleteData_longShort

In [ ]:
let portfolioMonthReturn (weights: Map<string,float>) (monthData: list<StockData>) =
    weights
    |> Map.toList
    |> List.map(fun (symbol, weight) ->
        let symbolData = 
            match monthData |> List.tryFind(fun x -> x.Symbol = symbol) with
            | None -> failwith $"You tried to find {symbol} in the data but it was not there"
            | Some data -> data
        symbolData.Return*weight)
    |> List.sum

In [ ]:
let weights_6040 = Map [("VTI",0.6);("BND",0.4)]

In [ ]:
let portMVE_long =
    stockDataByDateComplete_long
    |> List.map(fun (date, data) -> 
        { Symbol = "MVE Long Portfolio"
          Date = date
          Return = portfolioMonthReturn weights_long data })

let portMVE_longShort =
    stockDataByDateComplete_longShort
    |> List.map(fun (date, data) -> 
        { Symbol = "MVE Long-Short"
          Date = date
          Return = portfolioMonthReturn weights_longShort data })

// stockDataByDateComplete_long is used for 60-40 Portfolio
let port_6040 = 
    stockDataByDateComplete_long
    |> List.map(fun (date, data) -> 
        { Symbol = "60/40 Portfolio"
          Date = date 
          Return = portfolioMonthReturn weights_6040 data})

### Analyze the performance of your mean-variance efficient portfolios and your comparison diversified portfolio or portfolios.
Plot cumulative returns. You should make two graphs:
- one graph showing cumulative returns for the portfolios.
- one graph showing cumulative returns with a constant leverage
applied to each portfolio so that they all have an annualized
volatility of 10% over the full sample.

In [ ]:
let cumulateReturns (xs: list<StockData>) =
    let folder (prev: StockData) (current: StockData) =
        let newReturn = prev.Return * (1.0+current.Return)
        { current with Return = newReturn}
    
    match xs |> List.sortBy (fun x -> x.Date) with
    | [] -> []
    | h::t ->
        ({ h with Return = 1.0+h.Return}, t) 
        ||> List.scan folder

let portMveLongCumulative = 
    portMVE_long
    |> cumulateReturns

let portMveLongShortCumulative = 
    portMVE_longShort
    |> cumulateReturns

let port6040Cumulative = 
    port_6040
    |> cumulateReturns


let chartMVELong = 
    portMveLongCumulative
    |> List.map(fun x -> x.Date, x.Return)
    |> Chart.Line
    |> Chart.withTraceInfo(Name="MVE Long")

let chartMVELongShort = 
    portMveLongShortCumulative 
    |> List.map(fun x -> x.Date, x.Return)
    |> Chart.Line
    |> Chart.withTraceInfo(Name="MVE Long Short")

let chart6040 = 
    port6040Cumulative
    |> List.map(fun x -> x.Date, x.Return)
    |> Chart.Line
    |> Chart.withTraceInfo(Name="60/40")

let chartCombined =
    [ chartMVELong; chart6040;chartMVELongShort]
    |> Chart.combine
    |> Chart.withSize(900,600)
    |> Chart.withTitle("Diversified Portfolio - Cumulative Returns")

chartCombined

In [ ]:
portMVE_long
|> List.map(fun x -> x.Return)
|> Seq.stDev
|> fun vol -> sqrt(12.0) * vol

In [ ]:
portMVE_longShort
|> List.map(fun x -> x.Return)
|> Seq.stDev
|> fun vol -> sqrt(12.0)*vol


In [ ]:
port_6040
|> List.map(fun x -> x.Return)
|> Seq.stDev
|> fun vol -> sqrt(12.0)*vol

In [ ]:
let normalize10pctVol xs =
    let vol = xs |> List.map(fun x -> x.Return) |> Seq.stDev
    let annualizedVol = vol * sqrt(12.0)
    xs 
    |> List.map(fun x -> { x with Return = x.Return * (0.1/annualizedVol)})

let portMveCumulativeNormlizedVol_long = 
    portMVE_long
    |> normalize10pctVol
    |> cumulateReturns

let portMveCumulativeNormlizedVol_longshort = 
    portMVE_longShort
    |> normalize10pctVol
    |> cumulateReturns

let port6040CumulativeNormlizedVol = 
    port_6040
    |> normalize10pctVol 
    |> cumulateReturns


let chartMVENormlizedVol_long = 
    portMveCumulativeNormlizedVol_long
    |> List.map(fun x -> x.Date, x.Return)
    |> Chart.Line
    |> Chart.withTraceInfo(Name="MVE Long")

let chartMVENormlizedVol_longShort = 
    portMveCumulativeNormlizedVol_longshort
    |> List.map(fun x -> x.Date, x.Return)
    |> Chart.Line
    |> Chart.withTraceInfo(Name="MVE Long Short")

let chart6040NormlizedVol = 
    port6040CumulativeNormlizedVol
    |> List.map(fun x -> x.Date, x.Return)
    |> Chart.Line
    |> Chart.withTraceInfo(Name="60/40")

let chartCombinedNormlizedVol =
    [ chartMVENormlizedVol_long; chartMVENormlizedVol_longShort; chart6040NormlizedVol]
    |> Chart.combine
    |> Chart.withSize(900,600)
    |> Chart.withTitle("Diversified Portfolio - Cumulative Returns Levered")

chartCombinedNormlizedVol

Create a table to report performance measures for the portfolios over
the full period:
- What is their average annualized return? 
- What are their annualized Sharpe ratios?


In [ ]:
let avg_StockDataLong  = 
    let x = 
        portMVE_long
        |> List.map (fun x -> x.Return)
        |> List.map ( fun x -> annualizeMonthlyReturns x)
        |> List.average
    let y = 
        x * 100.0
    y

let avg_StockDataLongShort  = 
    let x = 
        portMVE_longShort
        |> List.map (fun x -> x.Return)
        |> List.map ( fun x -> annualizeMonthlyReturns x)
        |> List.average
    let y = 
        x * 100.0
    y

let avg_StockData_6040  = 
    let x = 
        port_6040
        |> List.map (fun x -> x.Return)
        |> List.map ( fun x -> annualizeMonthlyReturns x)
        |> List.average
    let y = 
        x * 100.0
    y

In [ ]:
let sharpe_StockData_Long = 
    portMVE_long
    |> List.map (fun x -> x.Return)
    |> Sharpe
    |> annualizeMonthlySharpe

let sharpe_StockData_LongShort = 
    portMVE_longShort
    |> List.map (fun x -> x.Return)
    |> Sharpe
    |> annualizeMonthlySharpe

let sharpe_StockData_6040 = 
    port_6040
    |> List.map (fun x -> x.Return)
    |> Sharpe
    |> annualizeMonthlySharpe

In [ ]:
let header = ["";"Avg Annualized Returns"; "Annualized Sharpe Ratio"]

let rows = [
                ["Long"; string avg_StockDataLong; string sharpe_StockData_Long ];
                ["Long-Short"; string avg_StockDataLongShort ; string sharpe_StockData_LongShort];
                ["60-40"; string avg_StockData_6040;string sharpe_StockData_6040 ];
]

Chart.Table(header, 
            rows) 
    |> Chart.withSize (800, 400)

In [ ]:
let portMveNormlizedVol_long = 
    portMVE_long
    |> normalize10pctVol 

let portMveNormlizedVol_longshort = 
    portMVE_longShort
    |> normalize10pctVol  

let port6040NormlizedVol = 
    port_6040
    |> normalize10pctVol 

let avg_StockDataLong_10  = 
    let x = 
        portMveNormlizedVol_long
        |> List.map (fun x -> x.Return)
        |> List.map ( fun x -> annualizeMonthlyReturns x)
        |> List.average
    let y = 
        x * 100.0
    y

let avg_StockDataLongShort_10  = 
    let x = 
        portMveNormlizedVol_longshort
        |> List.map (fun x -> x.Return)
        |> List.map ( fun x -> annualizeMonthlyReturns x)
        |> List.average
    let y = 
        x * 100.0
    y

let avg_StockData_6040_10  = 
    let x = 
        port6040NormlizedVol
        |> List.map (fun x -> x.Return)
        |> List.map ( fun x -> annualizeMonthlyReturns x)
        |> List.average
    let y = 
        x * 100.0
    y

let sharpe_StockData_Long_10 = 
    portMveNormlizedVol_long
    |> List.map (fun x -> x.Return)
    |> Sharpe
    |> annualizeMonthlySharpe

let sharpe_StockData_LongShort_10 = 
    portMveNormlizedVol_longshort
    |> List.map (fun x -> x.Return)
    |> Sharpe
    |> annualizeMonthlySharpe

let sharpe_StockData_6040_10 = 
    port6040NormlizedVol
    |> List.map (fun x -> x.Return)
    |> Sharpe
    |> annualizeMonthlySharpe

In [ ]:
let header = ["";"Avg Annualized Returns 10%"; "Annualized Sharpe Ratio 10%"]

let rows = [
                ["Long"; string avg_StockDataLong_10; string sharpe_StockData_Long_10 ];
                ["Long-Short"; string avg_StockDataLongShort_10 ; string sharpe_StockData_LongShort_10];
                ["60-40"; string avg_StockData_6040_10;string sharpe_StockData_6040_10 ];
]

Chart.Table(header, 
            rows) 
    |> Chart.withSize (800, 400)